In [1]:
import torch
import torchvision

import os
import numpy as np
from PIL import Image
from src.data.transforms import Compose, Resize, ToTensor
from collections import defaultdict
from tqdm import tqdm
import pandas as pd

In [2]:
jit_path = '/data/jit-checkpoints/basev3-freeze-jit-v6.pt'

In [3]:
model = torch.jit.load(jit_path)

In [4]:
base_path = '/workspace/pest-monitoring-new/deployment/sample_images/'
image_paths = os.listdir('sample_images/')

In [5]:
transform_pipeline = Compose(transforms = [Resize(512, 512), ToTensor()])

In [6]:
detections = defaultdict(list)
output = []
for i, path_ in enumerate(tqdm(image_paths)):
    path = os.path.join(base_path, path_)
    im_tensor = transform_pipeline(
        np.asarray(Image.open(path).convert("RGB")).astype(np.float32)
    )[0].unsqueeze(0)
    
    val_out, abw_boxes, pbw_boxes, abw_scores, pbw_scores = model(im_tensor)
    detections['path'].append(path_)
    detections['val_out'].append(val_out)
    detections['abw_boxes'].append(abw_boxes.shape[0])
    detections['pbw_boxes'].append(pbw_boxes.shape[0])
    detections['abw_scores'].append(abw_scores)
    detections['pbw_scores'].append(pbw_scores)

100%|███████████████████████████████████████████| 19/19 [00:05<00:00,  3.48it/s]


In [7]:
df = pd.DataFrame.from_dict(detections)[['path', 'abw_boxes', 'pbw_boxes']]

In [ ]:
df

In [ ]:
df_ = pd.read_csv('cfssd_2.5.csv')[['image_name', 'num_abw', 'num_pbw']]

In [ ]:
df_.merge(df, left_on = 'image_name', right_on = 'path')